# **Spatial Transcriptomics Deep Learning (STDL) Project Notebook**

> The notebook contains main experiments and examples of how to use the code

## **Phase 1: Pre-processing and technical preparations**

### 1.1: **Assign GPU device and allow CUDA debugging**

In [1]:
# create code to reimport module if i change it
%load_ext autoreload

In [2]:
# the next 2 lines are to allow debugging with CUDA !
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  
print(f'cuda debugging allowed')

cuda debugging allowed


In [3]:
%%time

import torch
print(f'cuda device count: {torch.cuda.device_count()}')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
#Additional Info when using cuda
if device.type == 'cuda':
    print(f'device name: {torch.cuda.get_device_name(0)}')
    print(f'torch.cuda.device(0): {torch.cuda.device(0)}')
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
# NOTE: important !!!!!!
# clearing out the cache before beginning
torch.cuda.empty_cache()

cuda device count: 1
Using device: cuda
device name: GeForce RTX 2080 Ti
torch.cuda.device(0): <torch.cuda.device object at 0x7fe980598a90>
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
CPU times: user 1.83 s, sys: 3.55 s, total: 5.37 s
Wall time: 3.23 s


### 1.2: **Import the Pre-Process module**

> `loadAndPreProcess` module contains methods to load the data files as pytorch and pandas objects, methods to preprocess the given data, and methods to create custom datasets from the preprocessed data.

<div class="alert alert-block alert-warning">
<b>TODO:</b> fill above line
</div>

In [4]:
# note: path to project is: /home/roy.rubin/STDLproject/
%autoreload 2
import loadAndPreProcess

### 1.3: **Load pytorch dataset objects from the image folder**

> loading regular and augmented datasets created from the given image folder with transformations.

> Note: `augmentedImageFolder` is a custom dataset of imageFolder objects with different transformations (see code).

> Note: `im_hight_and_width_size` will define the size to which the images in the folder will be resized to. their original size 176, and so if the number will be bigger, the images will be automaticaly upsampled in the `resize` (not sure by what method) - which means images might be "pixelized" / lower quality. The problem is, size 176 doesnt work with all models, so i had to increase the size.

In [5]:
im_hight_and_width_size = 176  # values: 176 (doesnt work with inception) / 224 (doesnt work with inception) / 299 (works with inception)

In [6]:
%%time

path_to_images_dir_patient1_train = "/home/roy.rubin/STDLproject/spatialGeneExpressionData/patient1/images"
imageFolder_train = loadAndPreProcess.load_dataset_from_images_folder(path_to_images_dir_patient1_train, im_hight_and_width_size)
augmentedImageFolder_train = loadAndPreProcess.load_augmented_imageFolder_DS_from_images_folder(path_to_images_dir_patient1_train, im_hight_and_width_size)


----- entered function load_dataset_from_pictures_folder -----

----- finished function load_dataset_from_pictures_folder -----


----- entered function load_dataset_from_pictures_folder -----

----- finished function load_dataset_from_pictures_folder -----

CPU times: user 223 ms, sys: 11.5 ms, total: 234 ms
Wall time: 303 ms


In [7]:
%%time

path_to_images_dir_patient2_test = "/home/roy.rubin/STDLproject/spatialGeneExpressionData/patient2/images"
imageFolder_test = loadAndPreProcess.load_dataset_from_images_folder(path_to_images_dir_patient2_test, im_hight_and_width_size)
# augmentedImageFolder_test = loadAndPreProcess.load_augmented_imageFolder_DS_from_images_folder(path_to_images_dir_patient2_test, im_hight_and_width_size) # not needed for now


----- entered function load_dataset_from_pictures_folder -----

----- finished function load_dataset_from_pictures_folder -----

CPU times: user 28.2 ms, sys: 1.04 ms, total: 29.3 ms
Wall time: 34.3 ms


### 1.4: **Load pandas dataframe objects from the given mtx/tsv/csv files**

> `matrix_dataframe` represents the gene expression count values of each sample for each gene

> `features_dataframe` contains the names of all the genes

> `barcodes_dataframe` contains the names of all the samples

In [8]:
%%time

path_to_mtx_tsv_files_dir_patient1_train = "/home/roy.rubin/STDLproject/spatialGeneExpressionData/patient1"
matrix_dataframe_train, features_dataframe_train , barcodes_dataframe_train = loadAndPreProcess.load_dataframes_from_mtx_and_tsv_new(path_to_mtx_tsv_files_dir_patient1_train)


----- entered function load_dataframes_from_mtx_and_tsv -----
started reading features.tsv
V  finished reading features.tsv
started reading barcodes.tsv
V  finished reading barcodes.tsv
started reading matrix.mtx. this might take some time ...
V  finished reading matrix.mtx
adjusting matrix_dataframe
V  finished working on matrix_dataframe

----- finished function load_dataframes_from_mtx_and_tsv -----

CPU times: user 1min 53s, sys: 1.6 s, total: 1min 54s
Wall time: 1min 59s


In [9]:
%%time

path_to_mtx_tsv_files_dir_patient2_test = "/home/roy.rubin/STDLproject/spatialGeneExpressionData/patient2"
matrix_dataframe_test, features_dataframe_test , barcodes_dataframe_test = loadAndPreProcess.load_dataframes_from_mtx_and_tsv_new(path_to_mtx_tsv_files_dir_patient2_test)


----- entered function load_dataframes_from_mtx_and_tsv -----
started reading features.tsv
V  finished reading features.tsv
started reading barcodes.tsv
V  finished reading barcodes.tsv
started reading matrix.mtx. this might take some time ...
V  finished reading matrix.mtx
adjusting matrix_dataframe
V  finished working on matrix_dataframe

----- finished function load_dataframes_from_mtx_and_tsv -----

CPU times: user 1min 44s, sys: 1.44 s, total: 1min 45s
Wall time: 1min 50s


### 1.5: **Remove samples from the matrix dataframe with no matching images in the image folder**

> Note: indices are being reset after this action, so a mapping of old to new column indices is returned: `column_mapping`.

> Note: the dataframe is also reordered according to the images order in the image folder

In [10]:
%%time

matrix_dataframe_train, column_mapping_train = loadAndPreProcess.cut_samples_with_no_matching_image_and_reorder_df(matrix_df=matrix_dataframe_train, 
                                                                                                                    image_folder_of_the_df=imageFolder_train, 
                                                                                                                    barcodes_df=barcodes_dataframe_train)

cutting samples that dont have mathching images in the image folder from the dataframe ...
V   done :)

CPU times: user 42 s, sys: 1.63 s, total: 43.6 s
Wall time: 45 s


In [11]:
%%time

matrix_dataframe_test, column_mapping_test = loadAndPreProcess.cut_samples_with_no_matching_image_and_reorder_df(matrix_df=matrix_dataframe_test, 
                                                                                                                  image_folder_of_the_df=imageFolder_test, 
                                                                                                                  barcodes_df=barcodes_dataframe_test)

cutting samples that dont have mathching images in the image folder from the dataframe ...
V   done :)

CPU times: user 43.1 s, sys: 1.41 s, total: 44.5 s
Wall time: 46.1 s


### 1.6: **Remove less-informative genes**

> we define *less-informative* genes as genes with less than K counts over all samples

> `Base_value` is a parameter for the user's choice

> Note: indices are being reset after this action, so a mapping of old to new column indices is returned: `row_mapping`.

In [12]:
# %%time

# Base_value = 10
# matrix_dataframe_train, row_mapping_train = loadAndPreProcess.cut_genes_with_under_B_counts(matrix_dataframe_train, Base_value) 

In [13]:
# %%time

# Base_value = 1
# matrix_dataframe_test, row_mapping_test = loadAndPreProcess.cut_genes_with_under_B_counts(matrix_dataframe_test, Base_value) 
# #TODO: note that this actually is not really needed, but it is to not change the functions inside this class to adjust to the test data

In [14]:
%%time

# begin by asserting that our dataframes have the same genes to begin with using the metadata of features_dataframe
assert features_dataframe_train['gene_names'].equals(features_dataframe_test['gene_names'])

Base_value = 10
matrix_dataframe_train, matrix_dataframe_test, row_mapping = loadAndPreProcess.cut_genes_with_under_B_counts_from_train_and_test(matrix_dataframe_train, matrix_dataframe_test, Base_value) 

checking for genes (rows) that contain less than B counts in both dataframes ...
discarding relevant rows ...
CPU times: user 54 s, sys: 2.22 s, total: 56.2 s
Wall time: 58.1 s


### 1.7: **Normalize matrix_dataframe entries**

> normaliztion will be performed on the remainning rows of the dataframe with the logic "log 1P"

> This method Calculates log(1 + x)

In [15]:
%%time

matrix_dataframe_train = loadAndPreProcess.perform_log_1p_normalization(matrix_dataframe_train) 

performing log1P transformation of the dataframe ...

CPU times: user 1.72 s, sys: 75.7 ms, total: 1.8 s
Wall time: 1.88 s


In [16]:
%%time

matrix_dataframe_test = loadAndPreProcess.perform_log_1p_normalization(matrix_dataframe_test) 

performing log1P transformation of the dataframe ...

CPU times: user 2.07 s, sys: 12.1 ms, total: 2.08 s
Wall time: 2.15 s


### 1.8: **Create custom datasets**

> Each custom dataset is tailored per task

> there are four tasks: single gene prediction, k gene prediction, all gene prediction using NMF dim. reduction, all gene prediction using AE dim. reduction

> For each of the above tasks 2 datasets were created:

>> A Dataset created from the TRAIN data WITHOUT augmentation (without image transformations)

>> A Dataset created from the TRAIN data WITH augmentation (with image transformations)

>> A Dataset created from the TEST data WITHOUT augmentation (without image transformations)

In [17]:
%%time
gene_name = 'MKI67'
custom_DS_SingleValuePerImg = loadAndPreProcess.STDL_Dataset_SingleValuePerImg(imageFolder=imageFolder_train, 
                                                               matrix_dataframe=matrix_dataframe_train, 
                                                               features_dataframe=features_dataframe_train, 
                                                               barcodes_dataframe=barcodes_dataframe_train, 
                                                               column_mapping=column_mapping_train,
                                                               row_mapping=row_mapping,
                                                               chosen_gene_name=gene_name)
custom_DS_SingleValuePerImg_augmented = loadAndPreProcess.STDL_Dataset_SingleValuePerImg(imageFolder=augmentedImageFolder_train, 
                                                               matrix_dataframe=matrix_dataframe_train, 
                                                               features_dataframe=features_dataframe_train, 
                                                               barcodes_dataframe=barcodes_dataframe_train, 
                                                               column_mapping=column_mapping_train,
                                                               row_mapping=row_mapping,
                                                               chosen_gene_name=gene_name)
custom_DS_SingleValuePerImg_test = loadAndPreProcess.STDL_Dataset_SingleValuePerImg(imageFolder=imageFolder_test, 
                                                               matrix_dataframe=matrix_dataframe_test, 
                                                               features_dataframe=features_dataframe_test, 
                                                               barcodes_dataframe=barcodes_dataframe_test, 
                                                               column_mapping=column_mapping_test,
                                                               row_mapping=row_mapping,
                                                               chosen_gene_name=gene_name)


----- entering __init__ phase of  STDL_Dataset_SingleValuePerImg -----

----- finished __init__ phase of  STDL_Dataset_SingleValuePerImg -----


----- entering __init__ phase of  STDL_Dataset_SingleValuePerImg -----

----- finished __init__ phase of  STDL_Dataset_SingleValuePerImg -----


----- entering __init__ phase of  STDL_Dataset_SingleValuePerImg -----

----- finished __init__ phase of  STDL_Dataset_SingleValuePerImg -----

CPU times: user 954 ms, sys: 40.3 ms, total: 995 ms
Wall time: 1.1 s


<div class="alert alert-block alert-info">
<b>Note:</b> inside the init phase of `STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance` class, K genes with the highest variance are chosen from matrix_dataframe, and they are the only genes that are kept for training and testing purposes
</div>

In [18]:
%%time

k = 10
custom_DS_KGenesWithHighestVariance = loadAndPreProcess.STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance(imageFolder=imageFolder_train, 
                                                                           matrix_dataframe=matrix_dataframe_train, 
                                                                           features_dataframe=features_dataframe_train, 
                                                                           barcodes_dataframe=barcodes_dataframe_train, 
                                                                           column_mapping=column_mapping_train,
                                                                           num_of_dims_k=k)
custom_DS_KGenesWithHighestVariance_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance(imageFolder=augmentedImageFolder_train, 
                                                                           matrix_dataframe=matrix_dataframe_train, 
                                                                           features_dataframe=features_dataframe_train, 
                                                                           barcodes_dataframe=barcodes_dataframe_train, 
                                                                           column_mapping=column_mapping_train,
                                                                           num_of_dims_k=k)
custom_DS_KGenesWithHighestVariance_test = loadAndPreProcess.STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance(imageFolder=imageFolder_test, 
                                                                           matrix_dataframe=matrix_dataframe_test, 
                                                                           features_dataframe=features_dataframe_test, 
                                                                           barcodes_dataframe=barcodes_dataframe_test, 
                                                                           column_mapping=column_mapping_test,
                                                                           num_of_dims_k=k)


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance -----
calculate variance of all columns from  matrix_dataframe - and choosing K genes with higest variance ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance -----
calculate variance of all columns from  matrix_dataframe - and choosing K genes with higest variance ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance -----
calculate variance of all columns from  matrix_dataframe - and choosing K genes with higest variance ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----

CPU times: user 27.1 s, sys: 2.06 s, total: 29.2 s
Wall time: 30 s


<div class="alert alert-block alert-info">
<b>Note:</b> inside the init phase of `STDL_Dataset_KValuesPerImg_LatentTensor_NMF` class, an NMF decompositionis performed on the matrix_dataframe object
</div>

In [19]:
%%time

k = 10
custom_DS_LatentTensor_NMF = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_NMF(imageFolder=imageFolder_train, 
                                                                           matrix_dataframe=matrix_dataframe_train, 
                                                                           features_dataframe=features_dataframe_train, 
                                                                           barcodes_dataframe=barcodes_dataframe_train, 
                                                                           column_mapping=column_mapping_train,
                                                                           num_of_dims_k=k)
custom_DS_LatentTensor_NMF_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_NMF(imageFolder=augmentedImageFolder_train, 
                                                                           matrix_dataframe=matrix_dataframe_train, 
                                                                           features_dataframe=features_dataframe_train, 
                                                                           barcodes_dataframe=barcodes_dataframe_train, 
                                                                           column_mapping=column_mapping_train,
                                                                           num_of_dims_k=k)
custom_DS_LatentTensor_NMF_test = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_NMF(imageFolder=imageFolder_test, 
                                                                           matrix_dataframe=matrix_dataframe_test, 
                                                                           features_dataframe=features_dataframe_test, 
                                                                           barcodes_dataframe=barcodes_dataframe_test, 
                                                                           column_mapping=column_mapping_test,
                                                                           num_of_dims_k=k)


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_LatentTensor_NMF -----
performing NMF decomposition on main matrix dataframe ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_LatentTensor_NMF -----
performing NMF decomposition on main matrix dataframe ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_LatentTensor_NMF -----
performing NMF decomposition on main matrix dataframe ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----

CPU times: user 25min 8s, sys: 16min 51s, total: 42min
Wall time: 1min 24s


<div class="alert alert-block alert-info">
<b>Note:</b> 
<ul>
  <li>first we create a dataset from `matrix_dataframe_train` to feed our AEnet.</li>
  <li>Then we create our AEnet and train it.</li>
  <li>Finally, we create our `custom_DS_LatentTensor_AE` class, in which the Autoencoder network will be saved.</li>
</ul>
</div>

In [ ]:
dataset_from_matrix_df = loadAndPreProcess.STDL_Dataset_matrix_df_for_AE_init(matrix_dataframe_train)

In [ ]:
%autoreload 2

from executionModule import get_Trained_AEnet
k = 10
AEnet = get_Trained_AEnet(dataset_from_matrix_df=dataset_from_matrix_df, z_dim=k, num_of_epochs=3, device=device)

In [ ]:
%%time

k = 10
custom_DS_LatentTensor_AE = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder(imageFolder=imageFolder_train, 
                                                                           matrix_dataframe=matrix_dataframe_train, 
                                                                           features_dataframe=features_dataframe_train, 
                                                                           barcodes_dataframe=barcodes_dataframe_train, 
                                                                           AEnet=AEnet,
                                                                           column_mapping=column_mapping_train,
                                                                           num_of_dims_k=k,
                                                                           device=device)
custom_DS_LatentTensor_AE_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder(imageFolder=augmentedImageFolder_train, 
                                                                           matrix_dataframe=matrix_dataframe_train, 
                                                                           features_dataframe=features_dataframe_train, 
                                                                           barcodes_dataframe=barcodes_dataframe_train, 
                                                                           AEnet=AEnet,                                                                                                            
                                                                           column_mapping=column_mapping_train,
                                                                           num_of_dims_k=k,
                                                                           device=device)
custom_DS_LatentTensor_AE_test = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder(imageFolder=imageFolder_test, 
                                                                           matrix_dataframe=matrix_dataframe_test, 
                                                                           features_dataframe=features_dataframe_test, 
                                                                           barcodes_dataframe=barcodes_dataframe_test, 
                                                                           AEnet=AEnet,                                                                                                       
                                                                           column_mapping=column_mapping_test,
                                                                           num_of_dims_k=k,
                                                                           device=device)

### 1.9: prepare for the next phases in which the experiments are executed

> import `executionModule` which contains the experiments, training methods, and testing methods

> create `hyperparameters` dictionary which will contain all of the hyper-parameters for our experiments (note - user can change these later)

> create `model_list` that will hold all the names for the models that will be used (only 3 models for now, as can be seen below). the models are:

>> `BasicConvNet` model

>> `DensetNet121` model

>> `Inception_V3` model

<div class="alert alert-block alert-warning">
<b>Warning:</b> change the hyper-parameters below with caution if needed !
</div>

In [21]:
%autoreload 2
import executionModule

# define hyperparameters for the experiments
hyperparameters = dict()
hyperparameters['batch_size'] = 25
hyperparameters['max_alowed_number_of_batches'] = 99999
hyperparameters['precent_of_dataset_allocated_for_training'] = 0.8
hyperparameters['learning_rate'] = 1e-4
hyperparameters['momentum'] = 0.9
hyperparameters['num_of_epochs'] = 3

# define hyperparameters for BsicConvNet
hyperparameters['channels'] = [32] 
hyperparameters['num_of_convolution_layers'] = len(hyperparameters['channels'])
hyperparameters['hidden_dims'] = [100]
hyperparameters['num_of_hidden_layers'] = len(hyperparameters['hidden_dims'])
hyperparameters['pool_every'] = 99999

# list of all models used
model_list = []
model_list.append('BasicConvNet')
model_list.append('DensetNet121')
# model_list.append('Inception_V3')  #TODO: still need to sort out input image size... !

<div class="alert alert-block alert-danger">
<b>Note:</b> inception_v3 model isnt sorted out yet... gives
    RuntimeError: Calculated padded input size per channel: (2 x 2). Kernel size: (5 x 5). Kernel size can't be greater than actual input size
</div>

> creating an assisting method for our testing that will time each experiment

In [22]:
def experiment_loop(ds_train, ds_test, phase_name):
    for model_name in model_list:
        print(f'\nstarting experiment **{model_name}**\n')
        %time executionModule.runExperiment(ds_train=ds_train, ds_test=ds_test, hyperparams=hyperparameters, device=device, model_name=model_name, dataset_name=phase_name)
        print(f'\nfinished experiment {model_name}')
    

## Phase 2: Single Gene Prediction

In [ ]:
experiment_loop(ds_train=custom_DS_SingleValuePerImg ,ds_test=custom_DS_SingleValuePerImg_test ,phase_name='single_gene')


starting experiment **BasicConvNet**


----- entered function runExperimentWithModel_BasicConvNet -----
/ * \ ENTERED train_prediction_model / * \ 
****** begin training ******

iteration 1 of 3 epochs


/home/roy.rubin/miniconda3/envs/cs236781-hw/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([25])) that is different to the input size (torch.Size([25, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


batch 152 of 152 batches
finished inner loop.
in this epoch: average loss 10.265031432193753

iteration 2 of 3 epochs
batch 152 of 152 batches
finished inner loop.
in this epoch: average loss 0.3465786508627628

iteration 3 of 3 epochs
batch 152 of 152 batches
finished inner loop.
in this epoch: average loss 0.30142253603001956
finished all epochs !
which means, that this model is now trained.
 \ * / FINISHED train_prediction_model \ * / 

----- entered function getSingleDimPrediction -----
--delete-- verify:  M_pred.shape (4015,)  ~  M_truth.shape (4015,)

----- finished function getSingleDimPrediction -----
TODO: print comparison of error results
recieved M_fast_reconstruction=None. errors with it will be 0
distance between M_truth, M_pred: 35.23397256734827
distance between M_truth, M_fast_reconstruction: 0
distance between M_pred, M_fast_reconstruction: 0

----- finished function runExperimentWithModel_BasicConvNet -----
CPU times: user 2min 28s, sys: 22.4 s, total: 2min 50s
Wall t

/home/roy.rubin/miniconda3/envs/cs236781-hw/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([25])) that is different to the input size (torch.Size([25, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


batch 152 of 152 batches
finished inner loop.
in this epoch: average loss 0.26913854156277683

iteration 2 of 3 epochs


In [ ]:
# experiment_loop(ds_train=custom_DS_SingleValuePerImg_augmented ,ds_test=custom_DS_SingleValuePerImg_test ,phase_name='single_gene_augmented')

## Phase 3: K genes prediction

In [ ]:
experiment_loop(ds_train=custom_DS_KGenesWithHighestVariance ,ds_test=custom_DS_KGenesWithHighestVariance_test ,phase_name='k_genes')

In [ ]:
# experiment_loop(ds_train=custom_DS_KGenesWithHighestVariance_augmented ,ds_test=custom_DS_KGenesWithHighestVariance_test ,phase_name='k_genes_augmented')

## Phase 4: All genes prediction - using dimensionality reduction techniques

### 4.1: Prediction using dimensionality reduction technique NMF

In [ ]:
experiment_loop(ds_train=custom_DS_LatentTensor_NMF ,ds_test=custom_DS_LatentTensor_NMF_test ,phase_name='NMF')

In [ ]:
# experiment_loop(ds_train=custom_DS_LatentTensor_NMF_augmented ,ds_test=custom_DS_LatentTensor_NMF_test ,phase_name='NMF_augmented')

### 4.2: Prediction using dimensionality reduction technique AE

In [ ]:
experiment_loop(ds_train=custom_DS_LatentTensor_AE ,ds_test=custom_DS_LatentTensor_AE_test ,phase_name='AE')

In [ ]:
# experiment_loop(ds_train=custom_DS_LatentTensor_AE_augmented ,ds_test=custom_DS_LatentTensor_AE_test ,phase_name='AE_augmented')

<div class="alert alert-block alert-danger">
<b>Note:</b> testing blocks
</div>

In [ ]:
# # temp
# print(custom_DS_SingleValuePerImg.features_dataframe)
# print('**')
# print(custom_DS_SingleValuePerImg_test.features_dataframe)
# print('**')
# print(custom_DS_SingleValuePerImg.features_dataframe['gene_names'].equals(custom_DS_SingleValuePerImg_test.features_dataframe['gene_names']))


In [ ]:
# #to_delete
# print(custom_DS_SingleValuePerImg.reduced_dataframe)
# print('**')
# print(custom_DS_SingleValuePerImg.reduced_dataframe.iloc[[2]].values)
# print('**')
# print(custom_DS_SingleValuePerImg.reduced_dataframe.iloc[2,0])

In [ ]:
# import numpy as np


# arr1 = np.array([[1, 1], [1, 1], [1, 1], [1, 1], [1, 1]], np.int32)  # shape: 5 batch size x 2 K
# print('******')
# print(arr1.shape)
# print(arr1)

# arr2 = np.array([[2, 2], [2, 2], [2, 2], [2, 2], [2, 2]], np.int32)  # shape: 5 batch size x 2 K
# print('******')
# print(arr2.shape)
# print(arr2)

# # Stack arrays in sequence vertically (row wise).

# stacked_once = np.vstack((arr1,arr2))
# print('******')
# print(stacked_once.shape)
# print(stacked_once)

# stacked_twice = np.vstack((stacked_once,arr2))
# print('******')
# print(stacked_twice.shape)
# print(stacked_twice)

In [ ]:
# import numpy as np
# import pandas as pd
# df = pd.DataFrame(np.random.randn(6, 6))
# print(df)
# print(f'***')
# ll = [2,4,5]
# df = df.iloc[:,ll]

# # hack
# mapping = df.T.reset_index(drop=False)
# mapping = mapping.rename(columns={"index": "original_index_from_matrix_dataframe"})  
# mapping = mapping['original_index_from_matrix_dataframe']
# # update columns' indices
# temp_len = len(df.columns)
# new_columns_index_list = list(range(temp_len))
# df.columns = new_columns_index_list
# print(mapping)
# print(f'***')
# print(mapping[1])
# print(f'***')
# print(df)
# print(f'***')

In [ ]:
# import numpy as np
# import pandas as pd
# df = pd.DataFrame(np.random.randn(6, 6))
# print(df)
# print(f'***')
# ll = []
# df = df.iloc[:,ll]
# print(df)
# print(f'***')
# # hack
# mapping = df.T.reset_index(drop=False)
# mapping = mapping.rename(columns={"index": "original_index_from_matrix_dataframe"})  
# mapping = mapping['original_index_from_matrix_dataframe']
# # update columns' indices
# temp_len = len(df.columns)
# new_columns_index_list = list(range(temp_len))
# df.columns = new_columns_index_list
# print(mapping)
# print(f'***')
# print(len(mapping.values))
# print(f'***')
# print(df)
# print(f'***')
# print(len(df.values))
# print(f'***')

In [ ]:
# ### test
# import pandas as pd
# # print(len(custom_DS_SingleValuePerImg))
# print(custom_DS_SingleValuePerImg.column_mapping)
# mmm = pd.DataFrame(data=custom_DS_SingleValuePerImg.column_mapping, columns=['original_index_from_matrix_dataframe']) # convert it from a pandas series to a pandas df
# print(mmm)
# print("***")
# print(mmm.index[mmm['original_index_from_matrix_dataframe']==3812])
# print("***")
# print(mmm.index[mmm['original_index_from_matrix_dataframe']==3812].tolist())
# print("***")
# print(mmm.index[mmm['original_index_from_matrix_dataframe']==3812].item())
# # row = self.row_mapping.index[self.row_mapping['original_index_from_matrix_dataframe'] == row_in_original_dataframe].tolist()

# # print(custom_DS_SingleValuePerImg[3812])